In [1]:
import pandas as pd

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "EleutherAI/pythia-70m-deduped"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/home/armin/Desktop/Projects/NLP/NLP-project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
#load that dataset
data = pd.read_csv('./Jfleg4-2-4.csv')


In [4]:
df_unique = data.drop_duplicates(subset=['input'])

In [ ]:
for sentence in df_unique['input']:
    sample = df_unique.sample(2)
    input_text = f""" I will give you a sentence which has grammatical errors. You have to give me the corrected version of that sentence. Look at this example:
    wrong sentence 1: {sample['input'].iloc[0]}
    The corrected version is: {sample['target'].iloc[0]}
    wrong sentence 2: {sample['input'].iloc[1]}
    The corrected version is: {sample['target'].iloc[1]}
    Now you have to correct the following sentence:
    {sentence}
    """
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(
        **inputs, 
        num_return_sequences=1, 
        temperature=0.9, 
        top_p=0.85, 
        pad_token_id=tokenizer.eos_token_id, 
        do_sample=True, 
        no_repeat_ngram_size=3, 
        # max_length=len(input_text) + len(sentence) + 10,
        max_length=len(input_text),
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Original sentence: {sentence}")
    print("*"*50)
    print(generated_text)
    print("#"*50)